In [106]:
from geopy.location import Location
from geopy.point import Point
from math import sqrt, acos, sin, cos, radians

In [40]:
from geopy.geocoders import Nominatim
import certifi
import ssl
import geopy.geocoders
from geopy import distance

ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
geopy.geocoders.options.default_user_agent = 'map'

geocoder = Nominatim()

In [48]:
from openrouteservice import convert

In [95]:
center_map = geocoder.geocode("Grenoble, France")
center_map

Location(Grenoble, Isère, Auvergne-Rhône-Alpes, France métropolitaine, France, (45.1875602, 5.7357819, 0.0))

In [186]:
demand = {"from_location": Location(address="40 rue Blanche Monier, Grenoble, France"),
          "to_location": Location(address="20 rue Lavoisier, Montbonnot-Saint-Martin, France")}
demand["from_location"] = geocoder.geocode(demand["from_location"].address)
demand["to_location"] = geocoder.geocode(demand["to_location"].address)
print("demand.from_location")
print(demand["from_location"].point)
print("({}, {})".format(demand["from_location"].point.latitude, demand["from_location"].point.longitude))
print("demand.to_location")
print(demand["to_location"].point)
print("({}, {})".format(demand["to_location"].point.latitude, demand["to_location"].point.longitude))

demand.from_location
45 11m 47.7557s N, 5 44m 17.912s E
(45.1965988, 5.7383089)
demand.to_location
45 13m 4.79028s N, 5 48m 41.1264s E
(45.2179973, 5.811424)


In [210]:
demand = {"from_location": Location(address="1 avenue Albert 1er de Belgique, Grenoble, France"),
          "to_location": Location(address="20 rue Lavoisier, Montbonnot-Saint-Martin, France")}
demand["from_location"] = geocoder.geocode(demand["from_location"].address)
demand["to_location"] = geocoder.geocode(demand["to_location"].address)
print("demand.from_location")
print(demand["from_location"].point)
print("({}, {})".format(demand["from_location"].point.latitude, demand["from_location"].point.longitude))
print("demand.to_location")
print(demand["to_location"].point)
print("({}, {})".format(demand["to_location"].point.latitude, demand["to_location"].point.longitude))

demand.from_location
45 10m 54.7738s N, 5 44m 2.5386s E
(45.1818816, 5.7340385)
demand.to_location
45 13m 4.79028s N, 5 48m 41.1264s E
(45.2179973, 5.811424)


In [211]:
offer = {"from_location": Location(address="26 rue Le Brix, Grenoble, France"),
          "to_location": Location(address="20 rue Lavoisier, Montbonnot-Saint-Martin, France")}
offer["from_location"] = geocoder.geocode(offer["from_location"].address)
offer["to_location"] = geocoder.geocode(offer["to_location"].address)
print("offer.from_location")
print(offer["from_location"].point)
print("({}, {})".format(offer["from_location"].point.latitude, offer["from_location"].point.longitude))
print("offer.to_location")
print(offer["to_location"].point)
print("({}, {})".format(offer["to_location"].point.latitude, offer["to_location"].point.longitude))

offer.from_location
45 10m 43.5068s N, 5 43m 22.0728s E
(45.1787519, 5.722798)
offer.to_location
45 13m 4.79028s N, 5 48m 41.1264s E
(45.2179973, 5.811424)


In [31]:
class solution:
    def __init__(self):
        self.x1 = -1
        self.y1 = -1
        self.x2 = -1
        self.y2 = -1
    
def pente_droite(a: Point, b: Point):
    if((b.x-a.x)==0):
        return 0.0
    return (b.y-a.y)/(b.x-a.x)

def origine_droite(a, pente):
    return a.y-pente*a.x

def solve(line_slope, line_origin, circle_origin, circle_radius, sol):
    # resoudre : line_slope x + line_origin = y et (x-circle_origin.x)²+(y-circle_origin.y)² = circle_radius
    cx = line_origin*line_origin+circle_origin.y*circle_origin.y-2*circle_origin.y*line_slope-2*circle_origin.y*line_origin+circle_origin.x*circle_origin.x-circle_radius
    ax = line_slope*line_slope+1
    bx = 2*line_slope*line_origin-2*circle_origin.x
    delta = bx*bx-4*ax*cx
    sol = solution()
    if(delta < 0):
        return false
    
    sol.delta = delta
    sol.x1 = (-bx-mth.sqrt(delta))/(2*ax)
    sol.y1 = line_slope*sol.x1 + line_origin
    sol.x2 = (-bx+mth.sqrt(delta))/(2*ax)
    sol.y2 = line_slope*sol.x2 + line_origin
    

def intersection(sol):
    return sol.delta >= 0
    
def is_between(a: point, b: point, c: point):
    # check if c is between a and b
    if a.x <= b.x:
        if c.x < a.x or c.x > b.x:
            return false
    else:
        if c.x < b.x or c.x > a.x:
            return false
    return true
        
    
    

In [148]:
import folium
from openrouteservice import client

api_key = '5b3ce3597851110001cf62486818cba49f3142778d9a9242848457fd'
clnt = client.Client(key=api_key)

In [212]:
params_route = {'profile': 'driving-car',
               'format_out': 'geojson',
               'geometry': 'true',
               'format': 'geojson',
               'instructions': 'false',
               }
params_route['coordinates'] = [[offer["from_location"].longitude,offer["from_location"].latitude],
                              [offer["to_location"].longitude,offer["to_location"].latitude]]
json_route = clnt.directions(**params_route)

json_route["features"][0]["geometry"]["coordinates"]

In [213]:
m = folium.Map(location=[center_map.latitude, center_map.longitude], zoom_start=12)
folium.features.GeoJson(json_route).add_to(m)

In [214]:
def perpendicular_projection(a: Point, b: Point, c: Point):
    if a == b:
        if a == c:
            return a, 0
        else:
            return (None, float('Infinity'))    
    else:
        lat_diff = b.latitude - a.latitude 
        lng_diff = b.longitude - a.longitude 
        ortogonal_shortest = ((lat_diff * (c.latitude - a.latitude)) + (lng_diff * (c.longitude - a.longitude))) / ((lat_diff * lat_diff) + (lng_diff * lng_diff))
        sol = Point(-1, -1)
        sol.latitude = a.latitude + lat_diff * ortogonal_shortest 
        sol.longitude = a.longitude + lng_diff * ortogonal_shortest
        if (sol.latitude >= min(a.latitude, b.latitude) and sol.latitude <= max(a.latitude, b.latitude) 
            and sol.longitude >= min(a.longitude, b.longitude) and sol.longitude <= max(a.longitude, b.longitude)) :
            d = acos(sin(radians(c.latitude)) * sin(radians(sol.latitude)) 
                     + cos(radians(c.latitude)) * cos(radians(sol.latitude)) 
                     * cos(radians(c.longitude - sol.longitude)))
            earth_radius = 6378137 # meters
            d = d * earth_radius
            #d = sqrt((c.latitude - sol.latitude) * (c.latitude - sol.latitude) +
            #(c.longitude - sol.longitude) * (c.longitude - sol.longitude))
            return sol, d    

    return (None, float('Infinity'))

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.plot([a.x, b.x], [a.y, b.y])
ax.plot(c.x, c.y, 'o')
if sol != None:
    ax.plot(sol.x, sol.y, 'o')

In [215]:
d_min = float('Infinity')
i_min = -1
s_min = Point()

for i in range(len(json_route["features"][0]["geometry"]["coordinates"]) - 1):
    a = Point(json_route["features"][0]["geometry"]["coordinates"][i][1], 
              json_route["features"][0]["geometry"]["coordinates"][i][0])
    b = Point(json_route["features"][0]["geometry"]["coordinates"][i+1][1], 
              json_route["features"][0]["geometry"]["coordinates"][i+1][0])
    #print(a.latitude)
    #print(json_route["features"][0]["geometry"]["coordinates"][i])
    sol, d = perpendicular_projection(a, b, demand["from_location"].point)
    #print(d)
    if d < d_min:
        d_min = d
        i_min = i
        s_min = sol

print("solution: ({}, {}), distance: {}".format(s_min.latitude, s_min.longitude, d_min))

solution: (45.18435528527839, 5.7330096220523), distance: 286.95889251153244


In [216]:
folium.map.Marker([demand["from_location"].latitude, demand["from_location"].longitude], icon=folium.Icon(color='blue',
                                        icon_color='#ffffff',
                                        icon='home'),
                      popup="demand from",
                 ).add_to(m) # Add apartment locations to map

if s_min is not None:
    folium.map.Marker([s_min.latitude, s_min.longitude], icon=folium.Icon(color='red',
                                        icon_color='#ffffff',
                                        icon='pushpin'
                                       ),
                      popup="meet here",
                 ).add_to(m) # Add apartment locations to map

In [217]:
m